# This is the code which solves the Static Traffic Assignment Problem

We want to solve the following problem (TAP-C):
\begin{align}
\min_{f, h} &\sum_{a} \int_{0}^{f_a} c_a(s)\; \text{d}s
\\
\text{s.t.  } & \;\; f = \Delta h
\\
& \;\; 
h \geq 0
\\
& \;\; 
A h = d
\\
& \;\; 
f \leq u
\end{align}

This problem is hard to compute, even if it is a convex problem. Computing all the path of a network is NP-hard (with respect to the number of edges and links).

Nevertheless, the TAP can be solve using a Frank-Wolf algorithm:
\begin{align}
&\text{1. TO DO }
\\
&\text{2. }
\end{align}


#### Remarks
First, we tried to solve the STA with CVX.
This does not work because the values of the travel time are to big. 

Secondly, we solve the STA using links flow. 
This does not help us in our project because we need the path flow to compute the dual of the TAP-C.

Thirdly, we solve the STA using paths flow. 
The issue here is that finding every path in a network in a NP-complete problem. 
But if we can compute every path (we only need to do it one time), then it is really fast to compute the shortest path. We do not need to do a Dikjstra's algorithm to compute the shortest path, only a big matrix multiplication and a array sorting are enough. This might be more difficult on a laptop. But it might be faster on a HPC.

Finally, we solve the STA using links flow and we compute the paths during the all or nothing step of the Frank Wolf algorithm.


In [1]:
debug = False

## 1. We load the graph and the demand
Both graph and demand are in csv file, we load them

In [2]:
import numpy as np
import scipy.sparse

### We clean the data

In [3]:
"""
We need to demand to be an array like:
[
[o1, d1, demand from o1 to d1],
...,
[on, dn, demand from on to dn]
]

We need the graph to be an array like:
[
[id link 1, node origin link 1, node destination link 1, a0, a1, a2, a3, a4],
...,
[id link n, node origin link n, node destination link n, a0, a1, a2, a3, a4]
]
where the node are indexed from 0 to nb_nodes-1,
    the links are indexed from 0 to nb_links-1,
    and a0, ..., a4 are the coeficient to calculate the travel time of one link as a function
    of the flow on the link: t = a0 + a1 * f + a2 * f**2 + a3 * f**3 + a4 * f**4

One can add some checks here to be sure that demand and graph respect this format!

Some demand and graph data can be found on:
    https://github.com/bstabler/TransportationNetworks
    Here one need to perprocess the data before using this code
"""

def cleaning_input(graph, demand):
    # in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix (2d array)
    try:
        demand.shape[1]
    except:
        demand = np.array([demand])
    nb_ods = int(demand.shape[0])

    # in the case where the index of the od pairs does not begin by 0, we rename the od pairs
    first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
    graph[:,1] = graph[:,1]-first_index_od
    graph[:,2] = graph[:,2]-first_index_od
    demand[:,0] = demand[:,0] - first_index_od
    demand[:,1] = demand[:,1] - first_index_od
    # WE SHOULD ALSO CHECK THAT EVERY DEMAND IS WELL DEFINED: >0
    return graph, demand

def load_network(network):
    graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
    demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)
    graph, demand = cleaning_input(graph, demand)
    if debug:
        print("The network " + network + " has been charged. The network characteristics are:")
        nb_links = int(np.max(graph[:,0])+1)
        nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
        nb_ods = int(demand.shape[0])
        print("\t nb nodes = " + str(nb_nodes))
        print("\t nb links = " + str(nb_links))
        print("\t nb ods = " + str(nb_ods))
    return graph, demand

debug_local = False
if debug_local:
    graph, demand = load_network(network_name)
    if network_name == Brae:
        demand[0][2] = 10

Then, we define the function which gives the travel time as a function of the flow

In [4]:
## Here Max float is fixed to not overlap the maximum float number when we compute the potential function
max_float = 1e+10
float_appro = 1e-3
if debug:
    print("The max capacity and to max travel time are " + str(max_float))

"""
We define the travel time of each link as a function of the flow on each link.
This function is given by the network topology (in the graph file) and by the capacity of each link.
One row of the graph is:
[id link 1, node origin link 1, node destination link 1, a0, a1, a2, a3, a4]
Each row correspond to one link.
The travel time of the link is t = a0 + a1 * f + a2 * f**2 + a3 * f**3 + a4 * f**4 if f < c 
    It is t = + inf (max_float) is f >= c
"""
def travel_time(graph, f, c = -1):
    try:
        if c == -1:
            c = [max_float for i in range(len(f))]
    except:
        ;
    # here we need to have the same indexation for graph and f. That why we need to store graph in a dictionnary
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    if len(c) != len(f):
        print("ERROR: in travel_time(graph, f, c = -1)\n******* the dimension of the capacity vector does not match the number of links *******")
    tt_tmp = [tt_tmp[i] if f[i]<c[i]-float_appro else max_float for i in range(len(f))]
    return tt_tmp

## 2. We compute the all or nothing flow allocation

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [5]:
"""
We store the travel time of each link in a sparce adjacency matrix G.
Given a flow allocation f and a capacities constraints c, we compute 
the travel time of each link and store it in G.

This allow us after to us the Disjtra's algorithm of the class scipy.sparse.csgraph
"""

def update_travel_time_from_tt(graph, tt, nb_nodes):
    G = np.zeros(shape=(nb_nodes,nb_nodes))
    G = scipy.sparse.lil_matrix(G)
    for i in range(graph.shape[0]):
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]
    G = G.tocsr()
    return G

def update_travel_time_from_flow(graph, f, nb_nodes, c = -1):
    return update_travel_time_from_tt(graph,travel_time(graph, f, c), nb_nodes)

debug_local = False
if debug_local:
    graph, _ = load_network(network_name)
    nb_links = int(np.max(graph[:,0])+1)
    nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
    G = update_travel_time_from_flow(graph, np.zeros(nb_links), nb_nodes) # , [1,1,-1,1,1]))
    print(G)

In [6]:
from scipy.sparse.csgraph import dijkstra

In [7]:
"""
Because the classic Frank Wolf algorithm does not take into account the paths,
we need to store the path in memory. We do it by using Object and a dictionnary 
called paths_used.

This idea is that we do not want to compute all the possible paths of the network
before the beginning of the algorithm. So we compute the paths used during the algorithm,
and we store them in memory.

We build at the same time the incidence matrix.

CAREFUL. We use the number of links of the graph in the calcul of the hash of every path.
This variable should be a parameter.
"""

class path:
    __nb_links = -1
    def set_nb_links(nb_l):
        path.__nb_links = nb_l
    def __init__(self,links):
        self.links = links
        self.__flow = 0
        if path.__nb_links == -1:
            raise Exception('The number of links as not be defined inside the path class. We need it for defining the hash of a link. Please use the function path.set_nb_links(nb_links) to define it')
    def set_flow(self, flow):
        self.__flow = flow
    def get_flow(self,):
        return self.__flow
    def __eq__(self, other):
        return np.all(self.links == other.links)
    # I am not sure about the hash table structure in Python. I am used to Java.
    def __hash__(self):
        return hash(np.sum([hash(self.links[i]*(path.__nb_links**i)) for i in range(len(self.links))]))
    def __str__(self):
        return str(self.__flow) + " is on " + str(self.links) 

debug_local = False
if debug_local:
    graph, _ = load_network(network_name)
    nb_links = int(np.max(graph[:,0])+1)
    path.set_nb_links(nb_links)
    paths_used = {}
    print(paths_used)
    p = path([0, 1, 2, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)
    p = path([0, 1, 1, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)
    p = path([0, 0, 2, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)

Now let's compute the all or nothing allocation

In [44]:
"""
This cell is the main point of the entire solver.
It is the all or nothing allocation.

"""


# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
def build_graph_adjacency(graph):
    graph_dict = {}
    for i in range(graph.shape[0]):
        try: 
            graph_dict[int(graph[i][1])]
        except:
            graph_dict[int(graph[i][1])] = {}
        graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    return graph_dict

# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
def build_demand_dict(demand):
    demand_dict = {}
    for i in range(demand.shape[0]):
        try: 
            demand_dict[int(demand[i][0])]
        except:
            demand_dict[int(demand[i][0])] = {}
        demand_dict[int(demand[i][0])][int(demand[i][1])] = i
    return demand_dict

def put_flow_on_short_path(faon, o_tmp, d_tmp, flow_tmp, return_predecessors, graph_dict, paths_used, paths_used_tmp, k, i, full_dijkstra):
    node_tmp = d_tmp
    links = []
    # using the dijkstra, we build the fastest path and we put the flow on it.
    while node_tmp != o_tmp:
        if debug_local:
            print(o_tmp)
            print(node_tmp)
        if not full_dijkstra:
            node_tmp_d = return_predecessors[i][node_tmp]
        else:
            node_tmp_d = return_predecessors[o_tmp][node_tmp]
        # Here we need the graph_dict to recover the link id from the nodes id.
        link_tmp = graph_dict[node_tmp_d][node_tmp]
        # we recover the path from the predecessor of the 
        links.insert(0, link_tmp)
        faon[link_tmp] += flow_tmp
        node_tmp = node_tmp_d

    p = path(links)
    p.set_flow(flow_tmp)


    if debug_local:
        print(p)
    # If we do not already know p, we add it to the delta matrix (here a dict)
    if not hash(p) in paths_used:
        if debug_local:
            print(k)
        # we add p and his index
        paths_used[hash(p)] = (p, k)
        paths_used_tmp[hash(p)] = (p, k)
        k = k+1
    else:
        # we recover the index of p using the paths_used dict
        paths_used_tmp[hash(p)] = (p, paths_used[hash(p)][1])
    return faon, paths_used, paths_used_tmp, k


# computing the all or nothing flow
def all_or_nothing(demand, G, graph_dict, paths_used, k, full_dijkstra = True):
    # k is the next index to give to a new path.
    
    debug_local = False
    # paths_used_tmp is the set of the path in the all or nothing allocation of this step
    paths_used_tmp = {}
    # faon is the flow allocation of the all or nothing allocation
    nb_links = G.nnz
    faon = np.zeros(nb_links)
    
    # using scipy to compute dijkstra
    """
    HERE ONE CAN CHANGE THE ALGORITHM TO BE FASTER. WE MIGHT REWRITE THE DIJKSTRA ALGORITHM.
    indices : array_like or int, optional
        if specified, only compute the paths for the points at the given indices.
    """
    indices = None
    if not full_dijkstra:
        indices = [int(demand[i,0]) for i in range(len(demand))]
        
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True, indices = indices)
    if debug_local:
        print(return_predecessors)
        print(indices)
        
    # for every origin destination pairs (i.e. one line of the demand file)
    nb_ods = int(demand.shape[0])
    for i in range(nb_ods):
        # we compute the shortest path and add the demand on it.
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]
        faon, paths_used, paths_used_tmp, k = put_flow_on_short_path(faon, o_tmp, d_tmp, flow_tmp, return_predecessors, graph_dict, paths_used, paths_used_tmp, k, i, full_dijkstra)

    return faon, paths_used_tmp, paths_used, k

We define the line search

In [45]:
debug_local = False
if debug_local:
    # we load the network
    graph, demand = load_network(network_name)
    nb_links = int(np.max(graph[:,0])+1)
    print(nb_links)
    path.set_nb_links(nb_links)
    nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
    G = update_travel_time_from_flow(graph, np.zeros(nb_links), nb_nodes)
    graph_dict = build_graph_adjacency(graph)
    paths_used = {}
    k = 0 
    _, _ , pp, k = all_or_nothing(demand, G, graph_dict, paths_used, k)
    for p in pp.values():
        print(p[0])
    all_or_nothing(demand, G, graph_dict, paths_used, k, False)
    for p in paths_used.values():
        print(p[0])

In [46]:
"""
The function potential is used to compute the line search between 
the all or nothing allocation and the current flow allocation
The function potential returns the objective function corresponding to
the flow allocation f.

The function line search does a 1D line search.
"""

def potential(graph, f, c=-1):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    try:
        if c == -1:
            c = [max_float for i in range(len(f))] # this might be to much, to do once we have everything done
    except:
        ;
    # here we need to have the same indexation for graph and f.
    pot_tmp = graph[:,3]*f + 1/2*graph[:,4]*(f**2) + 1/3*graph[:,5]*(f**3) + 1/4*graph[:,6]*(f**4) + 1/5*graph[:,7]*(f**5)
    pot_tmp = [pot_tmp[i] if f[i]<c[i] else f[i]*max_float for i in range(len(f))]
    return np.sum(pot_tmp)


def line_search(f, res=20):
    debug_local = False
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if debug_local:
            print(l * d, end=": ")
            print(f(l * d))
            print(r * d, end=": ")
            print(f(r * d))
            print(str(m1 * d) + " = " + str(f(m1 * d)))
            print(str(m2 * d) + " = " + str(f(m2 * d)))
            print()
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [47]:
def build_network(graph, c):
    nb_links = int(np.max(graph[:,0])+1)
    path.set_nb_links(nb_links)
    nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
    G = update_travel_time_from_flow(graph, np.zeros(nb_links), nb_nodes, c)
    graph_dict = build_graph_adjacency(graph)
    return nb_links, nb_nodes, G, graph_dict
    
def initialization_FW(demand, G, graph_dict, all_paths_used, k, graph, nb_nodes, c):
    debug_local = False
    
    f, paths_used_for_this_iter, all_paths_used, k = all_or_nothing(demand, G, graph_dict, all_paths_used, k)
    if debug_local:
        print(G)
        print(f)
    G = update_travel_time_from_flow(graph, f, nb_nodes, c)
    path_flow_matrix = np.zeros(k)
    for val in paths_used_for_this_iter.values():
        path_flow_matrix[val[1]] = val[0].get_flow()

    if debug:
        print("Test of initialization_FW")
        print(f)
        for p in paths_used_for_this_iter.values():
            print(p[0])
        print(path_flow_matrix)
    return f, paths_used_for_this_iter, all_paths_used, k, G, path_flow_matrix

def iteration_FW(demand, G, graph_dict, all_paths_used, k, graph, f, nb_nodes, c, path_flow_matrix, i):
    # WE COMPUTE THE ALL OR NOTHING ALGORITHM
    faon, paths_used_for_this_iter, all_paths_used, k = all_or_nothing(demand, G, graph_dict, all_paths_used, k)
    
    # we find the better convex combinaison of f and faon
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon, c))
    # HERE WE SHOULD BE CAREFUL IN THE CASE WHERE WE HAVE THE CAPACITY CONSTRAINTS
    # THE GRADIENT OF THE FUNCTION IS NOT THE SHORTEST PATH
    # WE SHOULD REMOVE THE PATH THAT SATURATED THE LINKS
    # AND THEN COMPUTE THE SOLUTION WITHOUT THE SATURATION, AND WITHOUT THE 
    # CORRESPONDING DEMAND
    bool_cap = (type(c) != int)
    if s==0 and bool_cap:
        update_flow = True
        
        demand_tmp = demand.copy()
        nb_links = int(np.max(graph[:,0])+1)
        _, delta, route2od = output_FW(all_paths_used, nb_links, graph, f, demand)
        path_at_capacity = set()
        for j in range(len(f)):
            if f[j] > c[j] - float_appro and f[j]<c[j]:
                tab = np.argwhere(delta.toarray()[:,j]==1)[:,0]
                for path_c in tab: path_at_capacity.add(path_c)
        # Path at capacity are all the path that saturate links

        # We don't care about the flow at the capacity
        for path_c in path_at_capacity:
            # find the row of the demand
            demand_tmp[route2od[path_c]][2] += -path_flow_matrix[path_c]

        # WE COMPUTE THE ALL OR NOTHING ALGORITHM
        faon, paths_used_for_this_iter, all_paths_used, k = all_or_nothing(demand_tmp, G, graph_dict, all_paths_used, k)

        # I add the flow of the saturated path to faon 
        for path_c in path_at_capacity:
            for j in np.where(delta[path_c].toarray()[0]==1):
                for jj in j:
                    faon[jj] += path_flow_matrix[path_c]

        # we find the better convex combinaison of f and faon
        s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon, c))
        for path_c in path_at_capacity:
            path_flow_matrix[path_c] = path_flow_matrix[path_c] /(1-s) 
    else:
        update_flow = False
    f = (1. - s) * f + s * faon
    G = update_travel_time_from_flow(graph, f, nb_nodes, c)

    # we multiply the previous path flow matrix by the coeficient of the line search
    path_flow_matrix = (1-s) * path_flow_matrix
    # we add the new path at the end of the path flow matrix.
    path_flow_matrix = np.append(path_flow_matrix, np.zeros(len(all_paths_used)-path_flow_matrix.shape[0]))
    # we add the all or nothing path flow (mulitply by the coeficient of the line search) to the path flow matrix.
    for val in paths_used_for_this_iter.values():
        # val[1] is the index of the path, val[0] is the path object
        path_flow_matrix[val[1]] += s * val[0].get_flow()

    if debug and i % (nb_iter / 10) == 0:
        print("Iteration: " + str(i))
        print("demand = " + str(demand_tmp))
        print("s: " + str(s))
        print("h: " +str(path_flow_matrix))
        print("f: " + str(f))
        print("The paths used at this iteration are:")
        for p in paths_used_for_this_iter.values():
            print(p[0])
    return path_flow_matrix, f, G, all_paths_used, k, s

def output_FW(all_paths_used, nb_links, graph, f, demand):
    # MAYBE I SHOULD SPLIT THE CODE HERE TO MAKE EASIER TEST
    # At the end we do some work to return a proper output
    nb_paths = len(all_paths_used.keys())
    delta = np.zeros(shape=(nb_paths, nb_links)) # nb_links should be a parameter
    route2od = [0 for _ in range(nb_paths)]# np.zeros(shape=nb_paths)
    delta = scipy.sparse.lil_matrix(delta)
    tt_f = np.array(travel_time(graph, f))
    
    demand_dict = build_demand_dict(demand)

    for p in all_paths_used.values():
        # here I can built route2od matrix at the same time
        try:
            links_tmp = p[0].links
            for l in links_tmp:
                delta[p[1],l] = 1
            route2od[p[1]] = demand_dict[int(graph[int(links_tmp[0])][1])][int(graph[int(links_tmp[-1])][2])]
        except:
            ;
    delta = delta.tocsr()
    # I should built the route2od matrix here,
    # route2od should be build from the all_paths_used_dict
    return tt_f, delta, route2od

def Frank_Wolf_solver(graph, demand_bis, eps, nb_iter, c=-1):
    ######### FIRST, WE INITIALIZE THE ALGORITHM #########
    # We initialize the number of paths_used to 0, 
    if type(c)!=int:
        c = np.array(c)
    k = 0
    all_paths_used = {}
    
    # we built the network as a matrix from the graph file
    nb_links, nb_nodes, G, graph_dict = build_network(graph, c)
    
    # The initialization step: we put all the demand on the fastest free flow travel time paths.
    f, paths_used_for_this_iter, all_paths_used, k, G, path_flow_matrix = initialization_FW(demand_bis, G, graph_dict, all_paths_used, k, graph, nb_nodes, c)

    ######### THEN, I RUN THE ITERATION OF THE FRANK-WOLF ALGORITHM #########
    for i in range(nb_iter):
        path_flow_matrix, f, G, all_paths_used, k, s = iteration_FW(demand_bis, G, graph_dict, all_paths_used, k, graph, f, nb_nodes, c, path_flow_matrix, i)
        if s < eps:
            break
    if debug:
        print(path_flow_matrix)
       
    ######### FINALLY, I WORK ON THE OUTPUT TO RETURN #########
    tt_f, delta, route2od = output_FW(all_paths_used, nb_links, graph, f, demand_bis)
    return path_flow_matrix, tt_f, delta.toarray(), route2od


In [48]:
def check_wardrop(j, demand, route2od, delta, path_flow_matrix, tt_f):
    f = delta.T @ path_flow_matrix
    tt_p = delta @ tt_f
    od = j
    tab = []
    for i in range(len(route2od)):
        if route2od[i] == od:
            tab.append(i)
    print(od)
    print(tab)
    print(tt_p[tab])

def od2route(route2od):
    od2route = {}
    for i in range(len(route2od)):
        if route2od[i] not in od2route.keys():
            od2route[route2od[i]] = set()
        od2route[route2od[i]].add(i)
    print(route2od)
    return od2route

In [51]:
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Siou2 = 'data/SiouxFalls_bis'
Brae = 'data/braess'

network_name = Siou2
eps=1e-8
nb_iter = 2000
graph, demand = load_network(network_name)
if network_name == Brae:
    demand[0][2] = 10

path_flow_matrix, tt_f, delta, route2od = Frank_Wolf_solver(graph, demand, eps, nb_iter) #, [11,11,2,11,11])

debug = True
if debug:
    nb_paths = len(path_flow_matrix)
    print(path_flow_matrix)
    print(nb_paths)
    print(tt_f)
    print(delta)
    print(delta @ tt_f)
    print(route2od)
    print(delta.shape)
    print(od2route(route2od))
    for od in od2route(route2od).keys():
        print("check wardrop for the od: " + str(od))
        check_wardrop(od, demand, route2od, delta, path_flow_matrix, tt_f)


[100.         100.         200.01712725 ...   6.43978041   6.42375039
   1.05609141]
1391
[ 6.00083501  4.0085859   6.00081657  6.60070134  4.00869315  4.27161331
  4.01984112  4.26981313  2.31738332  7.22762234  2.31574498 10.01787242
  9.67277697  6.57399015  9.99768038 14.82496386  5.50319906  2.06324648
 14.69215373  5.55375428 15.0336344  10.77578309  9.65396673 15.17539551
  5.71769236  5.68381745 12.20298968 13.81179312 20.23510183 16.30884503
  7.13764842 12.407229   13.73727806 13.84849679  4.0202294  13.59259359
  3.02354323  3.02285823 17.61278173 13.70109267 12.37489333  9.06546148
 13.72302782 12.2406174   4.33562511  9.0516876  10.72147595 20.08606057
  9.4645616   3.16618734 16.30836824  9.49092017  7.41284388  2.06228919
  3.16379009  4.26063034  4.32649259  7.43505105  9.50956336  4.25975473
  9.4526389   8.08321158  7.714824    8.16716681  4.19892761 11.74488881
  9.08465149  7.71403003  4.21213981 12.23662935  9.0767655  12.36529205
  3.72301817 17.65644069 11.913382

331
[328, 729]
[ 9.08465149 21.49316152]
check wardrop for the od: 332
332
[329, 730]
[21.44994354 21.45165883]
check wardrop for the od: 333
333
[330, 731, 1084]
[25.19696137 25.21183307 25.21011778]
check wardrop for the od: 334
334
[331, 732, 965, 1242]
[38.20228329 46.22654587 38.20729105 46.2231537 ]
check wardrop for the od: 335
335
[332, 733, 966]
[32.20144828 52.22736244 32.20645604]
check wardrop for the od: 336
336
[333, 734, 967, 1085, 1174, 1348, 1362]
[42.20761599 42.21795996 42.2159842  42.21097644 42.2145678  42.2129522
 42.21262375]
check wardrop for the od: 337
337
[334, 735, 968, 1175, 1349]
[37.93600268 37.94634665 37.94101044 37.94295449 37.94133889]
check wardrop for the od: 338
338
[335, 736, 969, 1176, 1350]
[35.61861936 35.62896333 35.62362712 35.62557117 35.62395557]
check wardrop for the od: 339
339
[336, 737, 970]
[25.60074694 45.62664371 25.6057547 ]
check wardrop for the od: 340
340
[337]
[5.22703657]
check wardrop for the od: 341
341
[338, 738]
[10.7757830

In [58]:
print(tt_f)
f = delta.T @ path_flow_matrix
print(f)

[ 6.00083501  4.0085859   6.00081657  6.60070134  4.00869315  4.27161331
  4.01984112  4.26981313  2.31738332  7.22762234  2.31574498 10.01787242
  9.67277697  6.57399015  9.99768038 14.82496386  5.50319906  2.06324648
 14.69215373  5.55375428 15.0336344  10.77578309  9.65396673 15.17539551
  5.71769236  5.68381745 12.20298968 13.81179312 20.23510183 16.30884503
  7.13764842 12.407229   13.73727806 13.84849679  4.0202294  13.59259359
  3.02354323  3.02285823 17.61278173 13.70109267 12.37489333  9.06546148
 13.72302782 12.2406174   4.33562511  9.0516876  10.72147595 20.08606057
  9.4645616   3.16618734 16.30836824  9.49092017  7.41284388  2.06228919
  3.16379009  4.26063034  4.32649259  7.43505105  9.50956336  4.25975473
  9.4526389   8.08321158  7.714824    8.16716681  4.19892761 11.74488881
  9.08465149  7.71403003  4.21213981 12.23662935  9.0767655  12.36529205
  3.72301817 17.65644069 11.91338227  3.76017424]
(76,)


In [60]:
solution = np.loadtxt(network_name + '_result.csv', delimiter=',', skiprows=1)
print(solution[:,1] - f)

[ 1.13893300e+03  2.45990706e+01  2.39600362e+01 -2.55304961e+01
  2.48643033e+03 -2.86560053e+01  4.22387575e+01  1.73009739e+03
 -2.86108549e+01 -1.04974144e+02  4.29951915e+02 -7.19585683e+00
 -1.81760485e+01  2.40508074e+01  8.43075013e+00 -3.28496522e+01
  5.84948096e+01 -6.43792179e+01  3.23582583e+01 -6.24998434e+01
  4.74552394e+01 -1.68997060e+01  3.91877306e+02 -4.61876945e+01
 -7.09010463e+01  3.63302352e+02  1.22782573e+02 -6.66587659e+01
 -2.57152909e+01 -2.01779078e-01  8.95245030e+02 -1.23322715e+02
 -4.01436146e+01 -3.94078942e+01  6.45316622e+02  3.89971596e+01
 -9.94049729e+01  5.82813666e+02  9.82815553e+00  3.35593765e+02
 -4.35651389e+01  5.76992930e+00  1.48196274e+02  4.16937883e+01
 -3.37720283e+01  2.76219815e+01  2.09302533e+01  2.57007175e+01
  1.44074459e+01 -6.32058404e+01 -6.25368981e-02 -7.05507743e+00
  1.09029569e+01  5.65990399e+01  4.74862556e+01 -2.29914594e+01
  3.24692387e+01 -1.04433608e+01 -2.00239181e+01  8.67889437e+00
  2.48371175e+01  6.08572

In [61]:
print(solution[:,2] - tt_f)

[-1.87712045e-05  1.04846736e-04  1.75497966e-05 -2.71030835e-02
 -1.06494382e-04 -2.21148060e-03  3.38033254e-04  1.45462819e-03
 -2.00921135e-03 -9.43218640e-02  1.32724626e-03 -1.96472088e-02
 -2.14662677e-02  2.55275205e-02  2.30221782e-02 -1.34008853e-01
  4.89613254e-02 -1.02079246e-03  1.32005785e-01 -5.23413144e-02
  1.41073117e-01 -4.63095624e-02  1.61878321e-02 -1.37526005e-01
 -3.51593097e-02  3.34259366e-02  2.02699772e-01 -8.94228399e-02
 -1.50291855e-01 -8.27874905e-04  8.53761319e-02 -2.03974469e-01
 -1.47050427e-01 -1.57211102e-01 -4.38917128e-04  1.42562061e-01
 -7.46683142e-04  6.21445116e-04  4.82259948e-02  1.41552380e-01
 -1.40554204e-01  1.38828315e-02  8.85326305e-02  1.33987458e-01
 -9.41303269e-03  3.64560762e-02  5.73356159e-02  1.50215133e-01
  3.68968890e-02 -2.72253268e-03 -3.51089766e-04 -1.80657544e-02
  2.37829206e-02  8.97191779e-04  2.04478266e-03 -1.25925702e-03
  9.03819735e-03 -2.27770141e-02 -5.04998533e-02  4.75337744e-04
  6.26105026e-02  8.24492